In [1]:
import numpy as np
import pandas as pd

In [2]:
handcoded = pd.read_csv("handcoded.csv")

In [3]:
data0 = pd.read_csv("fullweibo_1011.csv")

In [4]:
joined_data = handcoded.merge(data0[["weibo_id", "content_cleaned"]], on='weibo_id', how='left')
handcoded1=joined_data[["weibo_id","content","content_cleaned","Labeled"]]

In [5]:
handcoded1.to_csv("handcoded1.csv", encoding="utf-8-sig", index= False)

In [6]:
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
from torch._C import *
import torch
from torch import nn
from torch.optim import Adam

C:\Users\10292\anaconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = df['Labeled'].tolist()
        self.texts = [tokenizer(text, padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['content_cleaned']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

class BertClassifier(nn.Module):
    def __init__(self, dropout=0.5):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

def train(model, train_data, val_data, learning_rate, epochs):
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=10)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):
            total_acc_train = 0
            total_loss_train = 0   
            total_pre_rec_train = 0
            total_pre_train = 0
            total_rec_train = 0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)[:, 0]

                batch_loss = criterion(output, train_label.float())
                total_loss_train += batch_loss.item()

                output_label = torch.round(output)
                acc = (output_label == train_label).sum()
                total_acc_train += acc

                total_pre_rec_train += ((output_label == 1) & (train_label == 1)).sum()
                total_pre_train += output_label.sum()
                total_rec_train += train_label.sum()

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0
            total_pre_rec_val = 0
            total_pre_val = 0
            total_rec_val = 0
            with torch.no_grad():
                for val_input, val_label in val_dataloader:
                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)[:, 0]

                    batch_loss = criterion(output, val_label.float())
                    total_loss_val += batch_loss.item()

                    output_label = torch.round(output)
                    acc = (output_label == val_label).sum()
                    total_acc_val += acc

                    total_pre_rec_val += ((output_label == 1) & (val_label == 1)).sum()
                    total_pre_val += output_label.sum()
                    total_rec_val += val_label.sum()

            print(f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Train Precision: {total_pre_rec_train / total_pre_train: .3f} | Train Recall: {total_pre_rec_train / total_rec_train: .3f}')
            print(f'Epochs: {epoch_num + 1} | Valid Loss: {total_loss_val / len(val_data): .3f} | Valid Accuracy: {total_acc_val / len(val_data): .3f} | Valid Precision: {total_pre_rec_val / total_pre_val: .3f} | Valid Recall: {total_pre_rec_val / total_rec_val: .3f}')
            
            torch.save(model.state_dict(), "models/model_%d.pt" % (epoch_num + 1))

def evaluate(model, test_data):
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()
    
    answer = []
    total_acc_test = 0
    total_pre_rec_test = 0
    total_pre_test = 0
    total_rec_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)[:, 0]
            answer.append(output)

            output_label = torch.round(output)
            acc = (output_label == test_label).sum()
            total_acc_test += acc

            total_pre_rec_test += ((output_label == 1) & (test_label == 1)).sum()
            total_pre_test += output_label.sum()
            total_rec_test += test_label.sum()

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print(f'Test Precision: {total_pre_rec_test / total_pre_test: .3f}')
    print(f'Test Recall: {total_pre_rec_test / total_rec_test: .3f}')

    return answer

def predict(model, test_data):
    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()
    
    answer = []
    

    with torch.no_grad():
        a=0
        for test_input, test_label in test_dataloader:
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)[:, 0]
            answer.append(output)
            if (a%50==0):
                print(f'labelled {a*2} comments')
            a+=1
            

    return answer

In [8]:
handcoded1["content_cleaned"] = handcoded1["content_cleaned"].astype(str)


In [9]:
from sklearn.model_selection import train_test_split

In [12]:
df_data_train, df_data_test = train_test_split(
    handcoded1[["Labeled", "content_cleaned"]], test_size=0.3, random_state=1027)

df_data_train_train, df_data_train_val = np.split(df_data_train, [int(0.7*len(df_data_train))])

In [14]:
device = torch.device('cpu')

In [16]:
## Fine Tune the old bert using new data

model_10 = BertClassifier()
model_10.load_state_dict(torch.load("model_10.pt", map_location=device))

EPOCHS = 10
LR = 1e-6

train(model_10, df_data_train_train, df_data_train_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:09<00:00, 28.65s/it]


Epochs: 1 | Train Loss:  0.060 | Train Accuracy:  0.782 | Train Precision:  0.857 | Train Recall:  0.526
Epochs: 1 | Valid Loss:  0.054 | Valid Accuracy:  0.825 | Valid Precision:  0.800 | Valid Recall:  0.696


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:08<00:00, 28.54s/it]


Epochs: 2 | Train Loss:  0.047 | Train Accuracy:  0.830 | Train Precision:  0.833 | Train Recall:  0.702
Epochs: 2 | Valid Loss:  0.052 | Valid Accuracy:  0.841 | Valid Precision:  0.882 | Valid Recall:  0.652


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:39<00:00, 30.66s/it]


...

Epochs: 8 | Train Loss:  0.033 | Train Accuracy:  0.878 | Train Precision:  0.915 | Train Recall:  0.754
Epochs: 8 | Valid Loss:  0.052 | Valid Accuracy:  0.841 | Valid Precision:  0.882 | Valid Recall:  0.652


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [06:59<00:00, 27.97s/it]


Epochs: 9 | Train Loss:  0.033 | Train Accuracy:  0.891 | Train Precision:  0.936 | Train Recall:  0.772
Epochs: 9 | Valid Loss:  0.049 | Valid Accuracy:  0.841 | Valid Precision:  0.882 | Valid Recall:  0.652


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:39<00:00, 30.64s/it]


Epochs: 10 | Train Loss:  0.032 | Train Accuracy:  0.891 | Train Precision:  0.936 | Train Recall:  0.772
Epochs: 10 | Valid Loss:  0.047 | Valid Accuracy:  0.825 | Valid Precision:  0.833 | Valid Recall:  0.652


In [18]:
evaluate(model_10, df_data_test)

Test Accuracy:  0.778
Test Precision:  0.688
Test Recall:  0.423


[tensor([0.0908, 0.0689]),
 tensor([0.9274, 0.1219]),
 tensor([0.0816, 0.8519]),
 tensor([0.6288, 0.0935]),
 tensor([0.9747, 0.2040]),
 tensor([0.9022, 0.1536]),
 tensor([0.1414, 0.9445]),
 tensor([0.0956, 0.0775]),
 tensor([0.1451, 0.1783]),
 tensor([0.0930, 0.1340]),
 tensor([0.0840, 0.0893]),
 tensor([0.8459, 0.0827]),
 tensor([0.0974, 0.1912]),
 tensor([0.1157, 0.1353]),
 tensor([0.1814, 0.1645]),
 tensor([0.1114, 0.0471]),
 tensor([0.4069, 0.1608]),
 tensor([0.5322, 0.9803]),
 tensor([0.5279, 0.1378]),
 tensor([0.1891, 0.2213]),
 tensor([0.1493, 0.1096]),
 tensor([0.1453, 0.9078]),
 tensor([0.1059, 0.9610]),
 tensor([0.2236, 0.1142]),
 tensor([0.9418, 0.1362]),
 tensor([0.1290, 0.2116]),
 tensor([0.2305, 0.2697]),
 tensor([0.1936, 0.1797]),
 tensor([0.1731, 0.1505]),
 tensor([0.1337, 0.2676]),
 tensor([0.1500, 0.0864]),
 tensor([0.3246, 0.3097]),
 tensor([0.0862, 0.0589]),
 tensor([0.2266, 0.0778]),
 tensor([0.0801, 0.3262]),
 tensor([0.9481, 0.2108]),
 tensor([0.5837, 0.1777]),
 

In [19]:
# Test middle epoch model

model_t = BertClassifier()
model_t.load_state_dict(torch.load("models/model_4.pt", map_location=device))
evaluate(model_t, df_data_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test Accuracy:  0.767
Test Precision:  0.667
Test Recall:  0.385


[tensor([0.1734, 0.1486]),
 tensor([0.8860, 0.2517]),
 tensor([0.1131, 0.7243]),
 tensor([0.6767, 0.1544]),
 tensor([0.9544, 0.1218]),
 tensor([0.8980, 0.3667]),
 tensor([0.2461, 0.9214]),
 tensor([0.1810, 0.1267]),
 tensor([0.1399, 0.1486]),
 tensor([0.0942, 0.1152]),
 tensor([0.1193, 0.0883]),
 tensor([0.9369, 0.0775]),
 tensor([0.1854, 0.1152]),
 tensor([0.1087, 0.1949]),
 tensor([0.1608, 0.0925]),
 tensor([0.1537, 0.2201]),
 tensor([0.1782, 0.1228]),
 tensor([0.6583, 0.9546]),
 tensor([0.4861, 0.0848]),
 tensor([0.1127, 0.0639]),
 tensor([0.1972, 0.1856]),
 tensor([0.2083, 0.8777]),
 tensor([0.1178, 0.9537]),
 tensor([0.1842, 0.1006]),
 tensor([0.9308, 0.1073]),
 tensor([0.1030, 0.1399]),
 tensor([0.2167, 0.1483]),
 tensor([0.1748, 0.2287]),
 tensor([0.1277, 0.0983]),
 tensor([0.1464, 0.1707]),
 tensor([0.1612, 0.0956]),
 tensor([0.2283, 0.2120]),
 tensor([0.0697, 0.0883]),
 tensor([0.1648, 0.1096]),
 tensor([0.1030, 0.1941]),
 tensor([0.9524, 0.0625]),
 tensor([0.6927, 0.1494]),
 

In [20]:
handcoded1

,weibo_id,content,content_cleaned,Labeled
0,LhozEkDqZ,肉丝说，女人的内心就像深邃的海洋…多久可以忘掉一个人？大概需要用一生 ?原图?,肉丝 说 女人 的 内心 就 像 深邃 的 海洋 多久 可以 忘掉 一个 人 大概 需要 用...,0
1,LhADbodWL,#美国将向乌克兰运送防空导弹#面对着美国鼓吹战争，继续怂恿着双方扩大战争规模，@北京大学 王...,美国 将 向 乌克兰 运送 防空 导弹 面对 着 美国 鼓吹 战争 继续 怂恿 着 双方 扩...,1
2,LlXjeAacG,今天网红的水军，明显多了起来 赞[1768] 原文转发[23] 原文评论[162]转发理由...,今天 网红 的 水军 明显 多 了 起来 赞 1768 原文 转发 23 原文 评论 162...,1
3,Li7i9fhtw,#一起登月# 时尚博主「暴躁小七」摩登演绎黑色Icon Glance雪地靴，为早春注入前卫学...,一起 登 月 时尚 博主 暴躁 小七 摩登 演绎 黑色 Icon Glance 雪地 靴 为...,0
4,Lm8mkFSDA,比如要做用户中心一上来就说我要做活动中心，要做积分商城，要做导流宣传，要做banner营销，...,比如 要 做 用户 中心 一 上来 就 说 我 要 做 活动 中心 要 做 积分 商城 要 ...,0
...,...,...,...,...
295,LnMmR2dx5,台湾超商店员偷吃两颗茶叶蛋，被判刑三个月！茶叶蛋真的很金贵滴，大陆人吃不起，没想到台湾人也吃...,台湾 超商 店员 偷吃 两 颗 茶叶蛋 被 判刑 三 个 月 茶叶蛋 真的 很 金贵 滴 大...,1
296,LqPXr9yA1,我们这里全员核酸，单位里抽调防疫志愿者，本人有幸拿到了大白“体验券”（扫码大白），连体防护服...,我们 这里 全员 核酸 单位 里 抽调 防疫 志愿者 本人 有幸 拿到 了 大白 体验券 扫...,1
297,Llt9pouWi,塞尔维亚国家虽小，但志气高【#武契奇说永不会忘记北约所做的一切#】当地时间24日，#塞尔维亚...,塞尔维亚 国家 虽 小 但 志气 高 武契奇 说 永不 会 忘记 北约 所 做 的 一切 当...,0
298,Lp2lf9sAQ,《经济危机与世界王朝的改朝换代》第一部分，讲王朝更替的历史规律。第二部分，讲经济危机的成因及...,经济 危机 与 世界 王朝 的 改朝换代 第一 部分 讲 王朝 更替 的 历史 规律 第二 ...,0


In [24]:
# Shuffle Handcoded1 to tune again

handcoded2 = handcoded1.sample(frac = 1)

In [25]:
df_data_train, df_data_test = train_test_split(
    handcoded2[["Labeled", "content_cleaned"]], test_size=0.3, random_state=1027)

df_data_train_train, df_data_train_val = np.split(df_data_train, [int(0.7*len(df_data_train))])

## Fine Tune the old bert using new data

model_10_s = BertClassifier()
model_10_s.load_state_dict(torch.load("model_10.pt", map_location=device))

EPOCHS = 10
LR = 1e-6

train(model_10_s, df_data_train_train, df_data_train_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:07<00:00, 28.48s/it]


Epochs: 1 | Train Loss:  0.067 | Train Accuracy:  0.741 | Train Precision:  0.806 | Train Recall:  0.439
Epochs: 1 | Valid Loss:  0.043 | Valid Accuracy:  0.873 | Valid Precision:  0.846 | Valid Recall:  0.647


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:22<00:00, 29.48s/it]


Epochs: 2 | Train Loss:  0.051 | Train Accuracy:  0.796 | Train Precision:  0.765 | Train Recall:  0.684
Epochs: 2 | Valid Loss:  0.042 | Valid Accuracy:  0.841 | Valid Precision:  0.733 | Valid Recall:  0.647


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [06:53<00:00, 27.53s/it]


...

Epochs: 8 | Train Loss:  0.037 | Train Accuracy:  0.844 | Train Precision:  0.815 | Train Recall:  0.772
Epochs: 8 | Valid Loss:  0.038 | Valid Accuracy:  0.857 | Valid Precision:  0.750 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:29<00:00, 29.96s/it]


Epochs: 9 | Train Loss:  0.036 | Train Accuracy:  0.850 | Train Precision:  0.818 | Train Recall:  0.789
Epochs: 9 | Valid Loss:  0.042 | Valid Accuracy:  0.825 | Valid Precision:  0.667 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:12<00:00, 28.87s/it]


Epochs: 10 | Train Loss:  0.035 | Train Accuracy:  0.871 | Train Precision:  0.839 | Train Recall:  0.825
Epochs: 10 | Valid Loss:  0.037 | Valid Accuracy:  0.841 | Valid Precision:  0.733 | Valid Recall:  0.647


In [26]:
evaluate(model_10_s, df_data_test)

Test Accuracy:  0.778
Test Precision:  0.688
Test Recall:  0.688


[tensor([0.2542, 0.3633]),
 tensor([0.8051, 0.3023]),
 tensor([0.4084, 0.5466]),
 tensor([0.1036, 0.9349]),
 tensor([0.8970, 0.0911]),
 tensor([0.0670, 0.1122]),
 tensor([0.6330, 0.8812]),
 tensor([0.3017, 0.7720]),
 tensor([0.7634, 0.0687]),
 tensor([0.0544, 0.9176]),
 tensor([0.0916, 0.7522]),
 tensor([0.0393, 0.2039]),
 tensor([0.1659, 0.7979]),
 tensor([0.2002, 0.7384]),
 tensor([0.7978, 0.0820]),
 tensor([0.3404, 0.9350]),
 tensor([0.6377, 0.1182]),
 tensor([0.8640, 0.1132]),
 tensor([0.5781, 0.0829]),
 tensor([0.7060, 0.1341]),
 tensor([0.1234, 0.1517]),
 tensor([0.8637, 0.8648]),
 tensor([0.1414, 0.7998]),
 tensor([0.1945, 0.1661]),
 tensor([0.4921, 0.8812]),
 tensor([0.1717, 0.0486]),
 tensor([0.2222, 0.0839]),
 tensor([0.8356, 0.1011]),
 tensor([0.1056, 0.0626]),
 tensor([0.1268, 0.8234]),
 tensor([0.1801, 0.0963]),
 tensor([0.7850, 0.0561]),
 tensor([0.9267, 0.0476]),
 tensor([0.1003, 0.5772]),
 tensor([0.1523, 0.8291]),
 tensor([0.2404, 0.1206]),
 tensor([0.0487, 0.3158]),
 

In [27]:
train(model_10_s, df_data_train_train, df_data_train_val, LR, EPOCHS)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [06:42<00:00, 26.83s/it]


Epochs: 1 | Train Loss:  0.034 | Train Accuracy:  0.864 | Train Precision:  0.836 | Train Recall:  0.807
Epochs: 1 | Valid Loss:  0.039 | Valid Accuracy:  0.825 | Valid Precision:  0.667 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [06:58<00:00, 27.90s/it]


Epochs: 2 | Train Loss:  0.031 | Train Accuracy:  0.871 | Train Precision:  0.852 | Train Recall:  0.807
Epochs: 2 | Valid Loss:  0.041 | Valid Accuracy:  0.857 | Valid Precision:  0.750 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:21<00:00, 29.43s/it]


...

Epochs: 8 | Train Loss:  0.022 | Train Accuracy:  0.905 | Train Precision:  0.877 | Train Recall:  0.877
Epochs: 8 | Valid Loss:  0.042 | Valid Accuracy:  0.841 | Valid Precision:  0.706 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:02<00:00, 28.14s/it]


Epochs: 9 | Train Loss:  0.022 | Train Accuracy:  0.912 | Train Precision:  0.879 | Train Recall:  0.895
Epochs: 9 | Valid Loss:  0.044 | Valid Accuracy:  0.825 | Valid Precision:  0.688 | Valid Recall:  0.647


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:17<00:00, 29.17s/it]


Epochs: 10 | Train Loss:  0.020 | Train Accuracy:  0.918 | Train Precision:  0.881 | Train Recall:  0.912
Epochs: 10 | Valid Loss:  0.044 | Valid Accuracy:  0.825 | Valid Precision:  0.667 | Valid Recall:  0.706


In [28]:
evaluate(model_10_s, df_data_test)

Test Accuracy:  0.800
Test Precision:  0.733
Test Recall:  0.688


[tensor([0.0951, 0.1765]),
 tensor([0.9285, 0.4667]),
 tensor([0.3591, 0.5894]),
 tensor([0.0646, 0.7728]),
 tensor([0.8818, 0.0495]),
 tensor([0.0297, 0.0759]),
 tensor([0.6528, 0.9320]),
 tensor([0.2411, 0.6881]),
 tensor([0.8790, 0.0840]),
 tensor([0.0421, 0.8546]),
 tensor([0.1793, 0.9141]),
 tensor([0.0278, 0.0922]),
 tensor([0.1239, 0.8403]),
 tensor([0.3164, 0.9282]),
 tensor([0.9092, 0.0266]),
 tensor([0.3188, 0.9201]),
 tensor([0.4934, 0.0956]),
 tensor([0.9565, 0.0377]),
 tensor([0.3067, 0.1187]),
 tensor([0.9067, 0.3601]),
 tensor([0.0387, 0.0814]),
 tensor([0.9657, 0.9070]),
 tensor([0.0636, 0.8380]),
 tensor([0.0559, 0.0980]),
 tensor([0.5406, 0.9469]),
 tensor([0.3637, 0.0630]),
 tensor([0.1247, 0.0988]),
 tensor([0.6638, 0.0912]),
 tensor([0.1013, 0.0342]),
 tensor([0.0840, 0.9469]),
 tensor([0.1804, 0.0811]),
 tensor([0.7121, 0.0368]),
 tensor([0.9196, 0.0896]),
 tensor([0.0812, 0.9159]),
 tensor([0.1379, 0.8400]),
 tensor([0.3829, 0.1421]),
 tensor([0.0448, 0.1361]),
 

In [29]:
train(model_10_s, df_data_train_train, df_data_train_val, LR, EPOCHS)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [10:30<00:00, 42.00s/it]


Epochs: 1 | Train Loss:  0.021 | Train Accuracy:  0.925 | Train Precision:  0.883 | Train Recall:  0.930
Epochs: 1 | Valid Loss:  0.043 | Valid Accuracy:  0.841 | Valid Precision:  0.667 | Valid Recall:  0.824


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [12:44<00:00, 50.96s/it]


Epochs: 2 | Train Loss:  0.019 | Train Accuracy:  0.918 | Train Precision:  0.869 | Train Recall:  0.930
Epochs: 2 | Valid Loss:  0.046 | Valid Accuracy:  0.857 | Valid Precision:  0.722 | Valid Recall:  0.765


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [16:42<00:00, 66.86s/it]


Epochs: 3 | Train Loss:  0.019 | Train Accuracy:  0.918 | Train Precision:  0.881 | Train Recall:  0.912
Epochs: 3 | Valid Loss:  0.040 | Valid Accuracy:  0.841 | Valid Precision:  0.706 | Valid Recall:  0.706


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [11:45<00:00, 47.02s/it]


Epochs: 4 | Train Loss:  0.017 | Train Accuracy:  0.925 | Train Precision:  0.883 | Train Recall:  0.930
Epochs: 4 | Valid Loss:  0.047 | Valid Accuracy:  0.825 | Valid Precision:  0.667 | Valid Recall:  0.706


100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [25:52<00:00, 103.49s/it]

KeyboardInterrupt



In [30]:
# Test middle epoch model

model_t2 = BertClassifier()
model_t2.load_state_dict(torch.load("models_before_shuffle/model_4.pt", map_location=device))
evaluate(model_t2, df_data_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Test Accuracy:  0.856
Test Precision:  0.880
Test Recall:  0.688


[tensor([0.1215, 0.2981]),
 tensor([0.9586, 0.1058]),
 tensor([0.0724, 0.3096]),
 tensor([0.1521, 0.9604]),
 tensor([0.8214, 0.1201]),
 tensor([0.1217, 0.0573]),
 tensor([0.0631, 0.8358]),
 tensor([0.0669, 0.9636]),
 tensor([0.7600, 0.1232]),
 tensor([0.1374, 0.8665]),
 tensor([0.0768, 0.7722]),
 tensor([0.1060, 0.1472]),
 tensor([0.1451, 0.4162]),
 tensor([0.1140, 0.7701]),
 tensor([0.8668, 0.1463]),
 tensor([0.1450, 0.8633]),
 tensor([0.1339, 0.1482]),
 tensor([0.9362, 0.0833]),
 tensor([0.1026, 0.0947]),
 tensor([0.1458, 0.6356]),
 tensor([0.1995, 0.0836]),
 tensor([0.9540, 0.7988]),
 tensor([0.1310, 0.2175]),
 tensor([0.1706, 0.1411]),
 tensor([0.1557, 0.8868]),
 tensor([0.1417, 0.1208]),
 tensor([0.1579, 0.0863]),
 tensor([0.7105, 0.1926]),
 tensor([0.1365, 0.0771]),
 tensor([0.1419, 0.9611]),
 tensor([0.1688, 0.1459]),
 tensor([0.8543, 0.1220]),
 tensor([0.9152, 0.1791]),
 tensor([0.1450, 0.6131]),
 tensor([0.1692, 0.8402]),
 tensor([0.1437, 0.1200]),
 tensor([0.1020, 0.1345]),
 